In [1]:
using Random, Distributions, Statistics

In [2]:
large_class = Dict(:flat => (46,9,1),:med => (40,10,3), :gen => (34,11,5))
small_class = Dict(:flat => (23,4,1),:med => (19,5,2), :gen => (15,6,3))

Dict{Symbol,Tuple{Int64,Int64,Int64}} with 3 entries:
  :flat => (23, 4, 1)
  :gen  => (15, 6, 3)
  :med  => (19, 5, 2)

In [3]:
large_class = Dict(:flat => (46,9,1),:med => (40,10,3), :gen => (34,11,5))
small_class = Dict(:flat => (23,4,1),:med => (19,5,2), :gen => (15,6,3))

Dict{Symbol,Tuple{Int64,Int64,Int64}} with 3 entries:
  :flat => (23, 4, 1)
  :gen  => (15, 6, 3)
  :med  => (19, 5, 2)

In [12]:
sa = Dict()
sa[:A]=(:L1,:L6,:L12,:S1)
sa[:B]=(:L1,:L7,:L13,:S2)
sa[:C]=(:L2,:L7,:L9,:S3)
sa[:D]=(:L3,:L8,:L10,:S4)
sa[:E]=(:L4,:L8,:L12,:S5)
sa[:F]=(:L2,:L10,:L14,:S6)
sa[:G]=(:L4,:L6,:L14,:S7)
sa[:H]=(:L5,:L11,:L13,:S8)
sa[:I]=(:L3,:L11,:L15,:S9)
sa[:J]=(:L5,:L9,:L15,:S10)
section_assignments = sa

Dict{Any,Any} with 10 entries:
  :I => (:L3, :L11, :L15, :S9)
  :F => (:L2, :L10, :L14, :S6)
  :A => (:L1, :L6, :L12, :S1)
  :G => (:L4, :L6, :L14, :S7)
  :D => (:L3, :L8, :L10, :S4)
  :H => (:L5, :L11, :L13, :S8)
  :B => (:L1, :L7, :L13, :S2)
  :J => (:L5, :L9, :L15, :S10)
  :E => (:L4, :L8, :L12, :S5)
  :C => (:L2, :L7, :L9, :S3)

In [5]:
class_type = Dict()
for i=1:5
    class_type[Symbol("L",i)] = :crim
end

for i=6:8
    class_type[Symbol("L",i)] = :civ_pro
end

for i=9:11
    class_type[Symbol("L",i)] = :contracts
end

for i=12:15
    class_type[Symbol("L",i)] = :torts
end

for i in [1,2,5,7]
    class_type[Symbol("S",i)] = :contracts
end

for i in [3,4]
    class_type[Symbol("S",i)] = :torts
end

for i in [6,8,9,10]
    class_type[Symbol("S",i)] = :civ_pro
end

In [6]:
classes = foldl(vcat,map(collect,values(sa) |> collect)) |> union
sections = keys(sa) |> collect |> sort

10-element Array{Any,1}:
 :A
 :B
 :C
 :D
 :E
 :F
 :G
 :H
 :I
 :J

In [14]:
function generate_sample(tup, shuf = true)
    o1 = ones(tup[1]) * 3.3
    o2 = ones(tup[2]) * 3.7
    o3 = ones(tup[2]) * 3.0
    o4 = ones(tup[3]) * 4.0
    o5 = ones(tup[3]) * 2.7
    
    grades = foldl(vcat,[o1,o2,o3,o4,o5])
    
    out = shuf ? shuffle(grades) : sort(grades)
end
        

generate_sample (generic function with 2 methods)

In [7]:
function get_class_curves(classes::Array, tt_l,tt_s)
    class_curves = Dict()
    
    
    @assert tt_l |> isodd
    @assert tt_s |> iseven

    l_classes = filter(x->string(x)[1]=='L', classes) |> shuffle
    @assert length(l_classes) - 2 >= tt_l

    s_l = (length(l_classes) - tt_l)/2 |> x->(convert(Int,x))

    s_classes = filter(x->string(x)[1]=='S', classes) |> shuffle
    @assert length(l_classes) - 2 >= tt_s

    s_s = (length(s_classes) - tt_s)/2 |> x->(convert(Int,x))

    j = 1
    for i=1:tt_l
        class_curves[l_classes[i]] = :med
        j += 1
    end
    
    k = j
    for i=j:(j+ s_l - 1)
        class_curves[l_classes[i]] = :flat
        k += 1
    end
    
    for i=k:(k + s_l -1)
        class_curves[l_classes[i]] = :gen
    end
    
    j = 1
    for i=1:tt_s
        class_curves[s_classes[i]] = :med
        j += 1
    end
    
    k = j
    for i=j:(j+ s_s - 1)
        class_curves[s_classes[i]] = :flat
        k += 1
    end
    
    for i=k:(k + s_s -1)
        class_curves[s_classes[i]] = :gen
    end
    
    
    class_curves
end

get_class_curves (generic function with 1 method)

In [8]:
function run_sim(sa, class_types, shuffle_factor, tt_l, tt_s)
    classes = foldl(vcat,map(collect,values(sa) |> collect)) |> union
    
    students = Dict()
    all_students = []
    for s in keys(sa)
        arr = Vector{Symbol}()
        for i=1:33
            si = Symbol(string(s),i)
            push!(arr,si)
            push!(all_students, si)
        end
        students[s] = arr
    end
    student_grades = Dict()
    for s in all_students
        student_grades[s] = []
    end
    
    #normally distributed intelligence
    mm = 1
    dd = .1
    dist = Normal(mm,dd)
    
    student_smart = Dict()
    for s in all_students
        student_smart[s] = rand(dist)
    end
    
    ccc = Dict()
    for cl in classes
        ss = []
        for s in keys(sa) 
            if length(intersect(sa[s],[cl])) > 0
                push!(ss, s)
            end
        end
        ccc[cl] = ss
    end
    
    class_curves = get_class_curves(classes, tt_l, tt_s)
    
    out_test = []
    for cl in classes
        mult = class_type[cl] == :crim ? 3 : 4

        st = foldl(vcat,map(x->students[x], ccc[cl]))

        s_mod = []

        for s in st
            s_factor = (rand() - .5)*(2*shuffle_factor)
            push!(s_mod,(s,student_smart[s] + s_factor))
        end

        s_shuffled = sort(s_mod, by = x->x[2])
        this_curve = string(cl)[1] == 'L' ? large_class[class_curves[cl]] : small_class[class_curves[cl]]
        class_grades = generate_sample(this_curve, false) * mult

        out1 = []

        for (i,x) in enumerate(class_grades)
            push!(out1, (s_shuffled[i][1], x))
        end

        push!(out_test, out1)
    end  
    
    all_gp = foldl(vcat,out_test)
    
    for p in all_gp
        push!(student_grades[p[1]],p[2])
    end
    
    student_gpas = Dict()
    for k in keys(student_grades)
        student_gpas[k] = sum(student_grades[k])/15
    end
    
    student_gpas
end

run_sim (generic function with 1 method)

In [11]:
function full_results(results, full = false)
    gpas = values(results) |> collect
    for i =.05:.05:.95
        println("the ",i," percentile is: ", quantile(gpas, i))
    end
    if full
        return gpas |> sort
    end
end

full_results (generic function with 2 methods)

In [32]:
function x_percentile(results,x)
    gpas = values(results) |> collect
    quantile(gpas, x)
end

x_percentile (generic function with 1 method)

In [48]:
sims = Dict()
for i = 0:.05:.25
    mc = []
    for j=1:100
        out = run_sim(section_assignments, class_type, i, 13,8) |> x->x_percentile(x, .95)
        push!(mc, out)
    end
    sims[i] = mc
end

In [49]:
for k in sort(collect(keys(sims)))
    println("95th percentile for shuffle factor of ", k, " is: ", mean(sims[k]))
end

95th percentile for shuffle factor of 0.0 is: 3.8354999999999984
95th percentile for shuffle factor of 0.05 is: 3.820149999999994
95th percentile for shuffle factor of 0.1 is: 3.76969666666666
95th percentile for shuffle factor of 0.15 is: 3.712523333333331
95th percentile for shuffle factor of 0.2 is: 3.678483333333337
95th percentile for shuffle factor of 0.25 is: 3.6598700000000024
